In [121]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image

import examplesetup
import lib.deconvolution as deconvolution
import lib.utils as utils
import lib.xtrace as xtrace

from numpy.random import default_rng
rng = default_rng()

import tensorflow as tf


def random_image(dimensions):
    intensities = rng.random(dimensions)
    hit = rng.random(dimensions) > 0.1*rng.random()
    intensities[hit] = 0
    return intensities

def synthetic_depth_blur_psf(dimensions):
    dim = np.max(dimensions)
    
    config = {
    "detector": {
            "pixel_dims": np.array([1.0, 1.0, 3.0]),
            "mu":  0.3
        },
    "dimensions": np.array(dimensions),
    "ray_origin": np.array([dimensions[0]/3, dimensions[1]/3, dimensions[1]/3]),
    }
    # config = { #RANDOM CONFIG
    # "detector": {
    #         "pixel_dims": np.array([1.0, 1.0, 5*rng.random() + 0.1]),
    #         "mu":  rng.random()
    #     },
    # "dimensions": np.array(dimensions),
    # "ray_origin": np.array([
    #         dim*rng.normal() + dimensions[0]/2,
    #         dim*rng.normal() + + dimensions[1]/2,
    #         2*dim*rng.exponential() + 1
    #     ]),
    # }
    
    N = np.prod(dimensions)
    ray_grid = utils.random_ray_grid(dimensions)
    G = xtrace.depth_spill_psf(config, *ray_grid)
    return G

def apply_blur(img, G=None):
    if G is None:
        G = synthetic_depth_blur_psf(img.shape)
    distorted_img = (G.get()@img.flatten()).reshape(img.shape)
    distorted_img += 0.03*rng.poisson(10,img.shape)/10
    return distorted_img

dimensions = np.array([100, 100])
img = random_image(dimensions)
distorted_img = apply_blur(img)

fig, axs = plt.subplots(1, 2, figsize=(20, 10))
axs[0].imshow(img)
axs[1].imshow(distorted_img)

#https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3

plt.show()

ModuleNotFoundError: No module named 'tensorflow'